# Place for experimenting the progressive design

In [4]:
import yaml
import inspect
import importlib
import os,sys

sys.path.append('..')

import model_discovery.utils as U
from model_discovery.configs.gam_config import GAMConfig, GAMConfig_14M
from model_discovery.model.composer import GAUTree,GAUBase,GAUNode
# from model_discovery.evolution import BuildEvolution
import model_discovery.agents.flow.gau_utils
importlib.reload(model_discovery.agents.flow.gau_utils)
from model_discovery.agents.flow.gau_utils import check_and_reformat_gau_code

ckpt_dir = os.environ['CKPT_DIR']
lib_dir = U.pjoin(ckpt_dir, 'test_composer', 'lib')
test_tree = GAUTree('TestTree', None, None, None,None,lib_dir)

prompts_dir='../model_discovery/agents/prompts/'
gab_py = U.read_file(U.pjoin(prompts_dir,'gab_template.py'))
gam_py = U.read_file(U.pjoin(prompts_dir,'gam_prompt.py'))
GAU_TEMPLATE = U.read_file(U.pjoin(prompts_dir,'gau_template.py'))
GAU_BASE=inspect.getsource(GAUBase)


c:\Users\ChengJunyan1\anaconda3\envs\modis\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\ChengJunyan1\.cache\huggingface\token
Login successful


## Parsers

In [7]:
import json
import os,sys
import importlib

sys.path.append('..')

import model_discovery.agents.flow.gau_utils
importlib.reload(model_discovery.agents.flow.gau_utils)
from model_discovery.agents.flow.gau_utils import check_and_reformat_gau_code


code='''
# gau.py   # DO NOT CHANGE OR REMOVE THE MAKK HERE, KEEP IT ALWAYS THE FIRST LINE #

import torch
import torch.nn as nn

from model_discovery.model.utils.modules import GAUBase # DO NOT CHANGE THIS IMPORT STATEMENT #


# YOU CAN IMPORT MORE MODULES HERE #

# YOU CAN DEFINE MORE CLASSES OR FUNCTIONS HERE #

# Placeholder classes for future implementation
class MemoryAccessUnit(nn.Module):
    def __init__(self, embed_dim, memory_size, device=None, dtype=None):
        super().__init__(embed_dim)

    def _forward(self, X, **Z):
        return X, {}

class DownsamplingUnit(nn.Module):
    def __init__(self, embed_dim, downsample_factor, device=None, dtype=None):
        super().__init__(embed_dim)

    def _forward(self, X, **Z):
        return X, {}

class GAU(GAUBase): # DO NOT CHANGE THE NAME OF THIS CLASS
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all current intermediate variables}
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable
    """
    def __init__(self, embed_dim: int, device=None, dtype=None,**kwargs): # YOU CAN ADD MORE ARGUMENTS WITH OPTIONAL DEFAULT VALUES, BUT YOU HAVE TO HAVE embed_dim, device, dtype AS THE ARGUTMENTS #
        # argv: list of hyperparameters
        factory_kwargs = {"device": device, "dtype": dtype} # remember to pass it to all nn layers
        super().__init__(embed_dim) # DO NOT CHANGE THIS LINE #

        self.a1 = kwarg_all.get('a1', 1)
        self.a2: int = kwarg_all.get('a2', 2)

        # COMPLETING THE CODE HERE #
        self.token_scorer = TokenScoringGAU(embed_dim, **factory_kwargs)
        self.dual_path: GAUBase = DualPathGAU(embed_dim, **factory_kwargs)
        self.latent_attention: GAUBase = LatentAttentionGAU(embed_dim, **factory_kwargs)

        self.stages: nn.ModuleList = nn.ModuleList([CompressionStageGAU(embed_dim, device=
            device, dtype=dtype, **kwargs) for _ in range(2)])

        self.model = nn.Sequential(
          nn.Conv2d(1,20,5),
          nn.ReLU(),
          LatentAttentionGAU(),
          nn.Conv2d(20,64,5),
          nn.ReLU(),
          nn.Sequential(OrderedDict([
                ('conv1', nn.Conv2d(1, 20, 5)),
                ('relu1', nn.ReLU()),
                ('gau2', DualPathGAU()),
                ('conv2', nn.Conv2d(20, 64, 5)),
                ('relu2', nn.ReLU())
            ]))
        )

        self.activations = nn.ModuleDict({
            'relu': nn.ReLU(),
            'lrelu': nn.LeakyReLU(),
            'gau3': CompressionStageGAU()
        })

        self.model2 = nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(1,20,5)),
          ('relu1', nn.ReLU()),
          ('gau2', DualPathGAU())
          ('conv2', nn.Conv2d(20,64,5)),
          ('relu2', nn.ReLU())
        ]))

    # YOU CAN ADD MORE FUNCTIONS HERE #


    def _forward(self, X, **Z): 

        # THE CODE HERE MUST BE COMPLETED #
        # Step 1: Score tokens
        X, Z = self.token_scorer(X, **Z)
        # Step 2: Route through dual paths
        # Step 3: Apply latent attention
        Y, Z = self.latent_attention(X, **Z)
        X, Z = self.dual_path(X, **Z)

        return Y, Z
'''
children = ['TokenScoringGAU', 'DualPathGAU', 'LatentAttentionGAU','CompressionStageGAU'] # solve redundancy 

In [8]:

unit_name = "XAU"  # Provide the unit_name to rename GAU class
reformatted_code, new_args, errors, warnings = check_and_reformat_gau_code(code, unit_name,children)
print("Reformatted Code:\n" + reformatted_code)
print("Errors:\n", errors)
print("Warnings:\n", warnings)
print("New Arguments:\n", new_args)

# test_tree.add_unit(
#     unit_name,reformatted_code,new_args,None,called,None,None,children_units,None
# )
# test_tree.root=test_tree.units['XAU']

Rewritten instantiation: TokenScoringGAU(embed_dim, block_loc, kwarg_all, **self.factory_kwargs, **kwarg_all)
Rewritten instantiation: DualPathGAU(embed_dim, block_loc, kwarg_all, **self.factory_kwargs, **kwarg_all)
Rewritten instantiation: LatentAttentionGAU(embed_dim, block_loc, kwarg_all, **self.factory_kwargs, **kwarg_all)
Rewritten instantiation: CompressionStageGAU(embed_dim, block_loc, kwarg_all, **self.factory_kwargs, **kwarg_all)
Rewritten instantiation: CompressionStageGAU(embed_dim, block_loc, kwarg_all, **self.factory_kwargs, **kwarg_all)
Reformatted Code:
import torch
import torch.nn as nn
from model_discovery.model.utils.modules import GAUBase


class MemoryAccessUnit(nn.Module):

    def __init__(self, embed_dim, memory_size, device=None, dtype=None):
        super().__init__(embed_dim)

    def _forward(self, X, **Z):
        return X, {}


class DownsamplingUnit(nn.Module):

    def __init__(self, embed_dim, downsample_factor, device=None, dtype=None):
        super().

In [386]:
children={
    'unit1': 'MockUnit1',
    'unit2': 'MockUnit2',
}
called=['unit1', 'unit2', 'unit1']
test_tree.units['DualPathGAU'] = GAUNode('DualPathGAU', None, None, None, called, None, None, children, None)
children={
    'unit1': 'MockUnit3',
    'unit2': 'MockUnit4',
}
called=['unit2', 'unit2']
test_tree.units['MockUnit1'] = GAUNode('MockUnit1', None, None, None, called, None, None, children, None)

In [387]:
test_tree.units

{'DualPathGAU': GAUNode(name='DualPathGAU', code=None, args=None, desc=None, path=['unit1', 'unit2', 'unit1'], review=None, rating=None, children={'unit1': 'MockUnit1', 'unit2': 'MockUnit2'}, suggestions=None),
 'MockUnit1': GAUNode(name='MockUnit1', code=None, args=None, desc=None, path=['unit2', 'unit2'], review=None, rating=None, children={'unit1': 'MockUnit3', 'unit2': 'MockUnit4'}, suggestions=None)}

In [ ]:
class A:
    def __init__(self,r,d,e,**Z):
        print('Args in A:',d,e)
        print('Kwargs in A:',Z)

class B(A):
    def __init__(self,_Z,c=1,d=2,e=3,**Z):
        args2={'d':d,'e':e}
        super().__init__(c,**_Z,**args2)
        print('Kwargs in B:',Z)

cfg={
    'a':1,
    'b':2,
    'c':3,
}

B(cfg,**cfg)

Args in A: 2 3
Kwargs in A: {'a': 1, 'b': 2, 'c': 3}
Kwargs in B: {'a': 1, 'b': 2}


In [395]:
def A(X,Z,**kwargs):
    print(X)
    print(Z)
    print(kwargs)

def B(X,Z,a,**kwargs):
    A(X,Z,**Z)

Z={'a':3}
B(1,Z,**Z)

1
{'a': 3}
{'a': 3}


## Test


In [12]:
import os,sys
sys.path.append('..')
import torch
import torch.nn as nn
from model_discovery.model.utils.modules import GABBase


class GAB(GABBase):
    '''
    GAU Tree Map of Butterfly-AFT Generalized Autoregressive Block (BAGAB):
    ButterflyAFTGAU (Rating: 3.5/5)
        |- AFTMechanismGAU (Rating: 3.5/5)
            |- PositionBiasGAU (Rating: 3.5/5)
            |- ElementWiseOperationGAU (Rating: 3.5/5)
        |- ButterflyTransformGAU (Rating: 3.5/5)
            |- ButterflyLayerGAU (Rating: 3.5/5)
                |- ButterflyStageGAU (Rating: 3.5/5)
                    |- ButterflyParameterGAU (Unimplemented)
            |- ButterflyInitializationGAU (Rating: 3.5/5)
                |- ButterflyStageGAU (Rating: 3.5/5)
                    |- ButterflyParameterGAU (Unimplemented)
                |- ButterflyParameterGAU (Unimplemented)

    Implemented Units: AFTMechanismGAU, ButterflyLayerGAU, PositionBiasGAU, ButterflyTransformGAU, ElementWiseOperationGAU, ButterflyAFTGAU, ButterflyInitializationGAU, ButterflyStageGAU
    Unimplemented Units: AFTMechanismGAU, ButterflyLayerGAU, PositionBiasGAU, ButterflyTransformGAU, ButterflyParameterGAU, ElementWiseOperationGAU, ButterflyInitializationGAU, ButterflyStageGAU
    '''

    def __init__(self, embed_dim: int, block_loc: tuple, device=None, dtype
        =None, **kwargs):
        factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc)
        self.root = ButterflyAFTGAU(embed_dim=embed_dim, block_loc=
            block_loc, kwarg_all=kwargs, **factory_kwargs, **kwargs)

    def _forward(self, X, **Z):
        X, Z = self.root(X, **Z)
        return X, Z


from model_discovery.model.utils.modules import GAUBase


class ButterflyAFTGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.aft_mechanism = AFTMechanismGAU(embed_dim=embed_dim, block_loc
            =block_loc, kwarg_all=kwarg_all, **self.factory_kwargs, **kwarg_all
            )
        self.butterfly_transform = ButterflyTransformGAU(embed_dim=
            embed_dim, block_loc=block_loc, kwarg_all=kwarg_all, **self.
            factory_kwargs, **kwarg_all)

    def _forward(self, X, **Z):
        aft_output, Z = self.aft_mechanism(X, **Z)
        butterfly_output, Z = self.butterfly_transform(aft_output, **Z)
        return butterfly_output, Z


class ButterflyTransformGAU(GAUBase):

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.butterfly_layer = ButterflyLayerGAU(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all)
        self.butterfly_init = ButterflyInitializationGAU(embed_dim=
            embed_dim, block_loc=block_loc, kwarg_all=kwarg_all, **self.
            factory_kwargs, **kwarg_all)

    def _forward(self, X, **Z):
        X, Z = self.butterfly_init(X, **Z)
        Y, Z = self.butterfly_layer(X, **Z)
        return Y, Z


class ButterflyLayerGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.stages = nn.ModuleList([ButterflyStageGAU(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all) for _ in range(kwarg_all.get('num_stages', 4))])

    def _forward(self, X, **Z):
        for stage in self.stages:
            X, Z = stage(X, **Z)
        return X, Z

class ButterflyMatrixGAU(GAUBase):

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)

    def _forward(self, X, **Z):
        return X


import torch.nn.functional as F


class ButterflyInitializationGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.butterfly_size = kwarg_all.get('butterfly_size', embed_dim)
        self.butterfly_matrices = nn.ParameterList([nn.Parameter(torch.
            randn(self.butterfly_size, self.butterfly_size, **self.
            factory_kwargs)) for _ in range(int(torch.log2(torch.tensor(
            self.butterfly_size))))])
        self.butterfly_stage = ButterflyStageGAU(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all)
        self.butterfly_parameter = ButterflyParameterGAU(embed_dim=
            embed_dim, block_loc=block_loc, kwarg_all=kwarg_all, **self.
            factory_kwargs, **kwarg_all)

    def _forward(self, X, **Z):
        for matrix in self.butterfly_matrices:
            X = F.linear(X, matrix)
        return X, Z


class ButterflyParameterGAU(GAUBase):
    """
    Generalized Autoregressive Block Unit for learning parameters of Butterfly Factorization.
    Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
    Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
    Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

    embed_dim:    The dimension of the input embeddings
    block_loc:    The location of the block within the network, (layer_idx, n_block)
    kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.param_size = kwarg_all.get('param_size', 128)
        self.butterfly_weights = nn.Parameter(torch.randn(self.param_size,
            embed_dim, **self.factory_kwargs))
        nn.init.xavier_uniform_(self.butterfly_weights)

    def _forward(self, X, **Z):
        transformed_X = torch.matmul(X, self.butterfly_weights)
        Z_ = {'butterfly_weights': self.butterfly_weights}
        return transformed_X, Z_
        # transformed_X = F.linear(X, self.butterfly_weights.T)
        # return transformed_X, {}


class ButterflyComputationGAU(GAUBase):

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)

    def _forward(self, X, **Z):
        return X


class ButterflyStageGAU(GAUBase):

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.butterfly_params = ButterflyParameterGAU(embed_dim=embed_dim,
            block_loc=block_loc, kwarg_all=kwarg_all, **self.factory_kwargs,
            **kwarg_all)

    def _forward(self, X, **Z):
        transformed_X, Z = self.butterfly_params(X, **Z)
        assert transformed_X.shape == X.shape, f'Output shape {transformed_X.shape} does not match input shape {X.shape}'
        return transformed_X, Z

class AFTMechanismGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.position_bias = PositionBiasGAU(embed_dim=embed_dim, block_loc
            =block_loc, kwarg_all=kwarg_all, **self.factory_kwargs, **kwarg_all
            )
        self.element_wise_operation = ElementWiseOperationGAU(embed_dim=
            embed_dim, block_loc=block_loc, kwarg_all=kwarg_all, **self.
            factory_kwargs, **kwarg_all)

    def _forward(self, X, **Z):
        X, Z = self.position_bias(X, **Z)
        Y, Z = self.element_wise_operation(X, **Z)
        return Y, Z


class ElementWiseOperationGAU(GAUBase):
    """Generalized Autoregressive Block Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.position_bias = nn.Parameter(torch.zeros(1, embed_dim, **self.
            factory_kwargs))

    def _forward(self, X, **Z):
        queries = Z.get('queries', X)
        keys = Z.get('keys', X)
        values = Z.get('values', X)
        elementwise_product = queries * keys
        elementwise_sum = elementwise_product + values + self.position_bias
        return elementwise_sum, {}


class PositionBiasGAU(GAUBase):
    """Position Bias Generalized Autoregressive Unit
        Input:        X: (batch, seqlen, embed_dim), Z: {dict of all intermediate variables} 
        Output:       Y: (batch, seqlen, embed_dim), Z_: Optional, {dict of *new* intermediate variables to update the current Z}
        Constraints:  Causal, differentiable, parameter number, complexity, parallelizable...

        embed_dim:    The dimension of the input embeddings
        block_loc:    The location of the block within the network, (layer_idx, n_block)
        kwarg_all:    A dictionary of all hyperparameters across all units, use it instead of kwargs to initialize the children units
    """

    def __init__(self, embed_dim: int, block_loc: tuple, kwarg_all: dict,
        device=None, dtype=None, **kwargs):
        self.factory_kwargs = {'device': device, 'dtype': dtype}
        super().__init__(embed_dim, block_loc, kwarg_all)
        self.position_bias = nn.Parameter(torch.zeros(1, 1, embed_dim, **
            self.factory_kwargs))

    def _forward(self, X, **Z):
        Y = X + self.position_bias
        return Y, {}
    

class GABLM(nn.Module):
    def __init__(self, embed_dim, vocab_size=100, **kwargs):
        super().__init__()
        self.gab = GAB(embed_dim, (0,1), **kwargs)
        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)
    
    def forward(self, X, **Z):
        X, Z = self.gab(X, **Z)
        return self.lm_head(X),X



gab_config = {}


In [13]:
class SimpleGABLM(nn.Module):
    def __init__(self, embed_dim, vocab_size=100):
        super().__init__()
        self.butterfly_param = ButterflyParameterGAU(embed_dim, (0,1), {})
        self.lm_head = nn.Linear(embed_dim, vocab_size, bias=False)
    
    def forward(self, X, **Z):
        X, _ = self.butterfly_param(X, **Z)
        return self.lm_head(X), X

# Test the simplified model
simple_model = GABLM(128)
mock_input = torch.randn(1, 10, 128)
mock_y = torch.randint(0, 100, (1, 10))

optimizer = optim.Adam(simple_model.parameters())
criterion = nn.CrossEntropyLoss()

# Training loop
for _ in range(5):  # Run for a few iterations
    optimizer.zero_grad()
    y, X = simple_model(mock_input)
    loss = criterion(y.view(-1, 100), mock_y.view(-1))
    loss.backward()
    print(f"Loss: {loss.item()}")
    for name, param in simple_model.named_parameters():
        if param.grad is not None:
            print(f"Gradient for {name}: {param.grad.abs().mean()}")
        else:
            print(f"No gradient for {name}")
    optimizer.step()

Loss: 54495796.0
Gradient for gab.root.aft_mechanism.position_bias.position_bias: 899675.4375
Gradient for gab.root.aft_mechanism.element_wise_operation.position_bias: 475023.78125
Gradient for gab.root.butterfly_transform.butterfly_layer.stages.0.butterfly_params.butterfly_weights: 810216.875
Gradient for gab.root.butterfly_transform.butterfly_layer.stages.1.butterfly_params.butterfly_weights: 777344.75
Gradient for gab.root.butterfly_transform.butterfly_layer.stages.2.butterfly_params.butterfly_weights: 783032.0625
Gradient for gab.root.butterfly_transform.butterfly_layer.stages.3.butterfly_params.butterfly_weights: 771978.3125
Gradient for gab.root.butterfly_transform.butterfly_init.butterfly_matrices.0: 62701.796875
Gradient for gab.root.butterfly_transform.butterfly_init.butterfly_matrices.1: 66148.203125
Gradient for gab.root.butterfly_transform.butterfly_init.butterfly_matrices.2: 67731.2734375
Gradient for gab.root.butterfly_transform.butterfly_init.butterfly_matrices.3: 69984.

In [9]:
from torch import nn, optim 

model=GABLM(128, **gab_config)
vocab_size=100
DEFAULT_CONTEXT_LENGTH=10

mock_input=torch.randn(1, DEFAULT_CONTEXT_LENGTH, 128)
mock_y=torch.randint(0, 100, (1, 10))
mock_Z={}


criterion = nn.CrossEntropyLoss()
model.train()
optimizer = optim.Adam(model.parameters())

# Zero the parameter gradients
optimizer.zero_grad()
y,X=model(mock_input, **mock_Z)
loss=nn.CrossEntropyLoss()(y.view(-1, 100), mock_y.view(-1))
loss.backward()

used_params = set()

# Hook to register which parameters are used during the forward pass
def hook_fn(module, input, output):
    for param in module.parameters():
        used_params.add(param)

# Register hooks for all submodules
hooks = []
for submodule in model.modules():
    hook = submodule.register_forward_hook(hook_fn)
    hooks.append(hook)

# Forward pass again to collect used parameters
model(mock_input)

# Remove all hooks
for hook in hooks:
    hook.remove()

all_gradients_present = True

# Check gradients
for name, param in model.named_parameters():
    if param.requires_grad:
        if param in used_params and param.grad is None:
            # Used parameter without gradient -> Error
            print(f"Error: Used parameter {name} requires gradients but has none.")
            all_gradients_present = False
        elif param not in used_params:
            # Unused parameter -> Warning
            print(f"Warning: Parameter {name} was not used in the forward pass.")

if all_gradients_present:
    print('Differentiability test passed')
else:
    print('Differentiability test failed due to missing gradients.')


Error: Used parameter gab.root.butterfly_transform.butterfly_init.butterfly_stage.butterfly_params.butterfly_weights requires gradients but has none.
Error: Used parameter gab.root.butterfly_transform.butterfly_init.butterfly_parameter.butterfly_weights requires gradients but has none.
Differentiability test failed due to missing gradients.
